In [ ]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="quentinz/bge-large-zh-v1.5")

In [ ]:
from langchain_milvus import Milvus
from langchain_core.documents import Document
# The easiest way is to use Milvus Lite where everything is stored in a local file.
# If you have a Milvus server you can use the server URI such as "http://localhost:19530".
URI = "milvus_example.db"
vector_store = Milvus(
    embedding_function=embeddings,  # 替换为你的嵌入生成函数
    collection_name="my_collection2",
    connection_args={"uri": URI},
    auto_id=False,
    partition_names="sub_1",
    partition_key_field="source",
)

In [ ]:
from uuid import uuid4
documents = [
    Document(page_content="This is document 1.", metadata={"source": "doc1"}),
    Document(page_content="This is document 2.", metadata={"source": "doc2"}),
]
uuids = [str(uuid4()) for _ in range(len(documents))]
vector_store.add_documents(documents, ids=uuids)

In [ ]:
# 
results = vector_store.similarity_search("This is a query.", k=2)
for result in results:
    print(f"Content: {result.page_content}, Metadata: {result.metadata}")

In [ ]:
vector_store.delete(ids=[uuids[0]])

In [ ]:
# 删除collection
from pymilvus import MilvusClient
client = MilvusClient("./milvus_example.db")
print(client.describe_collection("my_collection"))
client.drop_collection(collection_name="my_collection")